In [1]:
# Importing Necessary Libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt

# Function to load model with error handling and compile with placeholder metrics
def load_and_compile_model(model_path):
    try:
        model = tf.keras.models.load_model(model_path)
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        print(f"Successfully loaded and compiled model from {model_path}")
        return model
    except Exception as e:
        print(f"Error loading model from {model_path}: {e}")
        return None

# Loading Pretrained Models
gender_model = load_and_compile_model(r"C:\Users\abhid\Downloads\gender_model.h5")
age_model = load_and_compile_model(r"C:\Users\abhid\Downloads\age_model.h5")
face_shape_model = load_and_compile_model(r"C:\Users\abhid\Downloads\face_shape_model.h5")

# Define the function for face detection and alignment
def detect_and_align_face(image_path):
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)
    
    # Assuming only one face per image for simplicity
    for (x, y, w, h) in faces:
        face = img[y:y+h, x:x+w]
        aligned_face = cv2.resize(face, (64, 64))
        return aligned_face
    return None

# Define function for gender prediction
def predict_gender(aligned_face):
    img_array = img_to_array(aligned_face) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    gender_pred = gender_model.predict(img_array)
    gender = 'male' if np.argmax(gender_pred) == 1 else 'female'
    return gender

# Define function for age prediction
def predict_age(aligned_face):
    img_array = img_to_array(aligned_face) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    age_pred = age_model.predict(img_array)
    age = np.argmax(age_pred)
    return age

# Define function for face shape prediction
def predict_face_shape(aligned_face):
    img_array = img_to_array(aligned_face) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    face_shape_pred = face_shape_model.predict(img_array)
    face_shape_classes = ['oval', 'round', 'square', 'heart', 'diamond']
    face_shape = face_shape_classes[np.argmax(face_shape_pred)]
    return face_shape

# Hairstyle recommendation based on face shape
def recommend_hairstyle(face_shape):
    recommendations = {
        'oval': 'Oval faces can try various hairstyles like long waves or sleek bobs.',
        'round': 'Round faces benefit from hairstyles with volume on top, like pompadours.',
        'square': 'Square faces look great with soft, layered styles or side-swept bangs.',
        'heart': 'Heart faces can try chin-length bobs or shoulder-length waves.',
        'diamond': 'Diamond faces look good with hairstyles that add width at the forehead, like side parts.'
    }
    return recommendations.get(face_shape, "No recommendation available")

# User Interface for Input and Display of Results
from IPython.display import display, Image as IPImage
from ipywidgets import widgets, VBox, HBox

def on_upload_change(change):
    for name, file_info in change['new'].items():
        # Save the uploaded file to disk
        with open(name, 'wb') as f:
            f.write(file_info['content'])
        
        # Display the uploaded image
        display(IPImage(name))
        
        # Process the image
        aligned_face = detect_and_align_face(name)
        if aligned_face is not None:
            gender = predict_gender(aligned_face)
            age = predict_age(aligned_face)
            face_shape = predict_face_shape(aligned_face)
            hairstyle_recommendation = recommend_hairstyle(face_shape)
            
            # Display the results
            results.value = f"""
            **Gender:** {gender}
            **Age:** {age}
            **Face Shape:** {face_shape}
            **Hairstyle Recommendation:** {hairstyle_recommendation}
            """
        else:
            results.value = "No face detected. Please upload a clearer image."

upload_button = widgets.FileUpload(accept='image/*', multiple=False)
upload_button.observe(on_upload_change, names='value')

results = widgets.HTML()

display(VBox([upload_button, results]))

# End of the Notebook


Successfully loaded and compiled model from C:\Users\abhid\Downloads\gender_model.h5
Successfully loaded and compiled model from C:\Users\abhid\Downloads\age_model.h5


Successfully loaded and compiled model from C:\Users\abhid\Downloads\face_shape_model.h5
